In [5]:
import glob
import os
import pickle

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
DATA_PATH="data/"

In [4]:
glob.glob(DATA_PATH + "*")

['data/all_reference_titles_20250204.txt',
 'data/all_reference_titles_not_in_llama_dedup_20250304.txt',
 'data/chunks_normas.pkl',
 'data/chunks_normas_new.pkl',
 'data/drive-download-20250122T172907Z-001.zip',
 'data/extracted_document_matches_20250210.pkl',
 'data/extracted_questions_20250203_final.pkl',
 'data/extracted_questions_20250203_parcial_01.pkl',
 'data/extracted_questions_20250203_parcial_02.pkl',
 'data/extracted_questions_20250203_parcial_03.pkl',
 'data/extracted_questions_20250203_parcial_04.pkl',
 'data/extracted_questions_20250207_final.pkl',
 'data/llama_deduplicated_20250204.txt',
 'data/llama_deduplicated_20250204_02.txt',
 'data/llama_deduplicated_reference_titles_20250205.txt',
 'data/llama_deduplicated_reference_titles_20250205_02.pkl',
 'data/llama_deduplicated_reference_titles_20250210.pkl',
 'data/llama_deduplicated_reference_titles_20250210_fixed.pkl',
 'data/normas_full.json',
 'data/P&R IRPF 2024 - v1.0 - 2024.05.03.pdf',
 'data/ragas_total_normas_acórdã

In [7]:
with open(DATA_PATH + "extracted_questions_20250207_final.pkl", "rb") as input_file:
    questions = pickle.load(input_file)

In [12]:
questions[559]

{'question_number': '560',
 'question_summary': 'ISENÇÕES DO GANHO DE CAPITAL',
 'question_text': 'Quais são as isenções relativas ao ganho de capital?',
 'answer': ['1 - Indenização da terra nua por desapropriação para fins de reforma agrária, conforme o disposto no § 5º do',
  'art. 184 da Constituição Federal de 1988;',
  'Atenção:',
  'A parcela da indenização, correspondente às benfeitorias, é computada como receita da atividade',
  'rural quando esta tiver sido deduzida como despesa de custeio ou investimento.',
  'A respeito de desapropriação, consulte a pergunta 633',
  '(Lei nº 7.713, de 22 de dezembro de 1998, art. 22, parágrafo único)',
  '2 - Indenização por liquidação de sinistro, furto ou roubo, relativo ao objeto segurado;',
  '(Lei nº 7.713, de 22 de dezembro de 1998, art. 22, parágrafo único)',
  '3 - Alienação, por valor igual ou inferior a R$ 440.000,00, do único bem imóvel que o titular possua,',
  'individualmente, em condomínio ou em comunhão, independentemente de

In [13]:
" ".join(questions[559]['answer_cleaned'])

'1 - Indenização da terra nua por desapropriação para fins de reforma agrária, conforme o disposto no § 5º do art. 184 da Constituição Federal de 1988; Atenção: A parcela da indenização, correspondente às benfeitorias, é computada como receita da atividade rural quando esta tiver sido deduzida como despesa de custeio ou investimento.   2 - Indenização por liquidação de sinistro, furto ou roubo, relativo ao objeto segurado;  3 - Alienação, por valor igual ou inferior a R$ 440.000,00, do único bem imóvel que o titular possua, individualmente, em condomínio ou em comunhão, independentemente de se tratar de terreno, terra nua, casa ou apartamento, ser residencial, comercial, industrial ou de lazer, e estar localizado em zona urbana ou rural, desde que não tenha efetuado, nos últimos cinco anos, outra alienação de imóvel a qualquer título, tributada ou não, sendo o limite considerado em relação: • à parte de cada condômino ou coproprietário, no caso de bens possuídos em condomínio; • ao imó

In [158]:
import json

from llm_access import *

import time

#### API keys
Expect to have a json file containing all the API keys needed. Groq key should be under "groq" identifier.

In [159]:
API_KEYS_FILE="/work/api_keys_20240427.json"

groq_key = json.load(open(API_KEYS_FILE))['groq']

#### Create the Groq model inteface to use

In [160]:
groq_llama33_70b_interface = groq_access(groq_key, GROQ_LLAMA3_3_70B_MODEL)

In [161]:
processing_start = time.time()

total_prompt_tokens = 0
total_completion_tokens = 0

for i, which_question in enumerate(questions[675 + 33:]):
    print("\n*****************************************")
    print(f"Processing question {i}\n")
    print("*****************************************\n")

    single_line_answer = " ".join(which_question['answer_cleaned'])
    
    result = embedded_legal_references_extraction(groq_llama33_70b_interface, single_line_answer)
    
    total_prompt_tokens += result['prompt_tokens']
    total_completion_tokens += result['completion_tokens']
    
    which_question['embedded_references'] = result['referências_citadas']
    
processing_end = time.time()


*****************************************
Processing question 0

*****************************************



Embedded references extraction. Answer size=496


Texto:
São tributados exclusivamente na fonte: 1 - à alíquota de 25%: • os benefícios líquidos resultantes da amortização antecipada, mediante sorteio, dos títulos de economia denominados capitalização; • os benefícios atribuídos aos portadores de títulos de capitalização nos lucros da empresa emitente. 2 - à alíquota de 20%: • nas demais hipóteses, inclusive no caso de resgate sem ocorrência de sorteio. 3 - à alíquota de 30%: • os prêmios em dinheiro, mediante sorteio, sem amortização antecipada. 

[{'role': 'system', 'content': 'Você recebe um texto que responde a uma pergunta no domínio jurídico e deve gerar uma lista incluindo somente os documentos jurídicos oficiais que são explicitamente referenciados no texto. Inclua detalhes de artigos, parágrafos, incisos, anexos ou qualquer outro detalhe incluído das referências. Igno

In [162]:
print(f"Total time to process the legal references: {processing_end - processing_start}")
print(f"total_prompt_tokens={total_prompt_tokens}, total_completion_tokens={total_completion_tokens}")

Total time to process the legal references: 4.275898218154907
total_prompt_tokens=2006, total_completion_tokens=93


In [163]:
with open("data/extracted_questions_with_embedded_references_20250212_before_formatting.pkl", "wb") as output_file:
    pickle.dump(questions, output_file, pickle.HIGHEST_PROTOCOL)

#### Print all extracted embedded references for a brief manual verification

In [166]:
for i, question in enumerate(questions):
    print(f"{i} = {question['embedded_references']}")

0 = ['Lei nº 11.196, de 21 de novembro de 2005, art. 39', 'Lei nº 14.754, de 12 de dezembro de 2023, art. 8º', 'Lei nº 14.754, de 2023, arts. 10 a 13', 'Lei nº 14.754, de 2023, art. 14', 'Instrução Normativa RFB nº 2.180, de 11 de março de 2024, art. 36', 'Instrução Normativa RFB nº 2.180, de 2024, arts. 41 a 47', 'Instrução Normativa RFB nº 2.180, de 2024, arts. 48 a 56']
1 = []
2 = []
3 = []
4 = []
5 = []
6 = []
7 = []
8 = []
9 = []
10 = []
11 = []
12 = []
13 = []
14 = []
15 = []
16 = []
17 = []
18 = []
19 = []
20 = []
21 = []
22 = []
23 = []
24 = ['Lei nº 9.532, de 10 de dezembro de 1997, art. 1º, § 1º', 'Instrução Normativa RFB nº 1.997, de 21 de novembro de 2019, art. 7º, inciso II', 'Código Tributário Nacional, art. 150, § 4º']
25 = []
26 = ['Instrução Normativa RFB nº 2.066, de 24 de fevereiro de 2022']
27 = []
28 = ['Declaração do Imposto sobre a Renda Retido na Fonte (Dirf)', 'Declaração de Serviços Médicos e de Saúde (Dmed)', 'Declaração de informações sobre Atividades Imobil